# Making pool of values for DOE - BO experiment

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# Length = [28.639, 28.276, 31.121, 19.835, 33.654, 30.594, 29.529]#np.arange(12,51)
# Width = [9.008, 9.004, 7.358, 14.762, 6.926, 8.897, 8.737]#np.arange(6,26)
Length = [25.59635, 27.39105, 23.54675, 27.3818]#np.arange(12,51)
Width = [5.9023, 7.48505, 12.7644, 8.75165]#np.arange(6,26)
Thickness = 1.22 #nm average(?)
Unit_volume = 0.221445125 #nm^3
epsilon = 31947800  # (1/M*cm)ROUGH ESTIMATE from JPCC 2015, 119, 26768		 *for 4 ML NPLs with lateral size of ~300nm^2	
Cd_atoms_per_unit_volume = 4
Se_atoms_per_unit_volume = 0.875
Ag_acet_concentration = [0.006,0.0007] #M
Dilution_factor_for_OD = 100
Ag_percentage_doping = [2,5,10]  # % of doping
Optical_density_at_dilution_factor_100 = [0.15,0.3]  #Experimentally measured  
Aliquote = [0.25,0.5] #mL of NPL Aliquote for experiments from Ag_acet_concentration
Mass_Ag_acet = [3.5,7.8,14] #mg Ag_acet
Ag_Acet_mw = 166.91 #g/mol  



In [2]:
# Create nested loops to iterate through the possible values of each parameter
import pandas as pd 
from tqdm import tqdm
from itertools import product
data = { 'Dimension1(nm)': [], 'Dimension2(nm)': [],  'Dilution(%)': [], 'Nano_stock_vol(mL)': [], 
        'Diluted_vol(mL)': [], 'AgConc(M)': [], 'AgMass(mg)': [], 'MeOH_vol(mL)': [], 
        'H2O_vol(mL)': [], 'Doping(%)': [], 'AgSol(uL)': [],'Temperature(C)': [],'Time(min)':[]}
Time = [10, 60, 180] # min
Temperature = [0, 25, 50] # C
areas = []
#get combinations of L and W without considering order
# L_w_combs = list(product(Length, Width))
L_w_combs = [(L,W) for (L, W) in zip(Length, Width)]
print(L_w_combs)
depurated_L_w_combs = []
areas = []
for L, W in L_w_combs:
    if L*W not in areas:
        areas.append(L*W)
        depurated_L_w_combs.append((L,W))

total_iterations = len(depurated_L_w_combs) * len([Dilution_factor_for_OD]) * len(Optical_density_at_dilution_factor_100) * len(Aliquote) * len(Ag_acet_concentration) * len(Mass_Ag_acet) * len(Ag_percentage_doping)
with tqdm(total=total_iterations) as pbar:
    for DF in [Dilution_factor_for_OD]:
        for OD in Optical_density_at_dilution_factor_100:
            for NSV in Aliquote:
                for AAC in Ag_acet_concentration:
                    for AM in Mass_Ag_acet:
                        for DP in Ag_percentage_doping:
                            for L, W in depurated_L_w_combs:
                                    # Calculations based on spreadsheet
                                    pbar.update(1)

                                    Unique_unit_areas = L * W
                                    Unique_unit_volumes = Unique_unit_areas * Thickness  # nm^3
                                    Unique_number_qd = Unique_unit_volumes / Unit_volume  # NUMBER OF QD per unit

                                    Concentration_at_dilution = OD / epsilon  # [NPL] M at dilution
                                    Concentration = Concentration_at_dilution * DF  # [NPL] M
                                    Cd_concentration = Concentration * Unique_number_qd * Cd_atoms_per_unit_volume # [Cd] M= [NPL] M * qd_number per NPL * Cd_atoms_per_unit_volume

                                    # Solution of ag_acet to get Ag_mol of 0.006M or 0.0007M from Ag_acet mass
                                    Ag_total_sol_volume = (AM / Ag_Acet_mw) / AAC  # mL
                                    Me_OH_vol = Ag_total_sol_volume * 5 / 7  # mL MeOH
                                    Water_vol = Ag_total_sol_volume * 2 / 7  # mL H2O
                                    Cd_mols = Cd_concentration * NSV / 1000
                                    Ag_mols_needed = Cd_mols * DP / 100  # mol of Ag needed
                                    Vol_Ag_acet_needed = Ag_mols_needed / AAC * 1000 * 1000 #micro liters
                                    # Append values to lists
                                    for T in Temperature:
                                        for t in Time:
                                            data['Dimension1(nm)'].append(L)
                                            data['Dimension2(nm)'].append(W)
                                            data['Dilution(%)'].append(OD)    #wrong name, it should be OD w dilution
                                            data['Nano_stock_vol(mL)'].append(NSV)
                                            data['Diluted_vol(mL)'].append(NSV*7)
                                            data['AgConc(M)'].append(AAC)
                                            data['AgMass(mg)'].append(AM)
                                            data['MeOH_vol(mL)'].append(Me_OH_vol)
                                            data['H2O_vol(mL)'].append(Water_vol)
                                            data['Doping(%)'].append(DP)
                                            data['AgSol(uL)'].append(Vol_Ag_acet_needed)
                                            data['Time(min)'].append(t)
                                            data['Temperature(C)'].append(T)

data = pd.DataFrame(data)
display(data)

[(25.59635, 5.9023), (27.39105, 7.48505), (23.54675, 12.7644), (27.3818, 8.75165)]


100%|██████████| 288/288 [00:00<00:00, 80881.12it/s]


,Dimension1(nm),Dimension2(nm),Dilution(%),Nano_stock_vol(mL),Diluted_vol(mL),AgConc(M),AgMass(mg),MeOH_vol(mL),H2O_vol(mL),Doping(%),AgSol(uL),Temperature(C),Time(min)
0,25.59635,5.90230,0.15,0.25,1.75,0.0060,3.5,2.496355,0.998542,2,1.302633,0,10
1,25.59635,5.90230,0.15,0.25,1.75,0.0060,3.5,2.496355,0.998542,2,1.302633,0,60
2,25.59635,5.90230,0.15,0.25,1.75,0.0060,3.5,2.496355,0.998542,2,1.302633,0,180
3,25.59635,5.90230,0.15,0.25,1.75,0.0060,3.5,2.496355,0.998542,2,1.302633,25,10
4,25.59635,5.90230,0.15,0.25,1.75,0.0060,3.5,2.496355,0.998542,2,1.302633,25,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,27.38180,8.75165,0.30,0.50,3.50,0.0007,14.0,85.589325,34.235730,10,354.207616,25,60
2588,27.38180,8.75165,0.30,0.50,3.50,0.0007,14.0,85.589325,34.235730,10,354.207616,25,180
2589,27.38180,8.75165,0.30,0.50,3.50,0.0007,14.0,85.589325,34.235730,10,354.207616,50,10
2590,27.38180,8.75165,0.30,0.50,3.50,0.0007,14.0,85.589325,34.235730,10,354.207616,50,60


In [3]:
print(len(data))

2592


In [4]:
data.to_csv('Data/pool_processed.csv', index=False)

In [ ]:
def make_prompt_from_features(features,version=0):    
    if version == 0:
        prompt_template = "Stock nanoplatelets have dimensions {Dimension1_nm} nm by {Dimension2_nm} nm and the optical density of 100x hexanes dilution of stock is {Dilution_pct}% " \
                        "recorded at 512nm. {Nano_stock_vol_mL} mL nanoplatelets stock is diluted 7x by hexanes using {Diluted_vol_mL} mL for each trial of doping. " \
                        "Silver acetate solution of {AgConc_M} M is made with {AgMass_mg} mg of silver acetate in {MeOH_vol_mL} mL of MeOH and {H2O_vol_mL} mL of water. " \
                        "{Doping_pct} % Ag doping requires {AgSol_uL} uL of silver doping solution, respectively. " \
                        "The reaction was performed at 1000 rpm for {Time_min} minutes at {Temperature_C} oC. " \
                        "Fluorescence of each doped sample was collected as a 30x dilution with hexanes."
    elif version == 1:
        prompt_template = "Stock nanoplatelets have dimensions {Dimension1_nm} nm by {Dimension2_nm} nm and the optical density of 100x hexanes dilution of stock is {Dilution_pct}% "\
                        "recorded at 512nm. {Nano_stock_vol_mL} mL nanoplatelets stock is diluted 7x by hexanes using {Diluted_vol_mL} mL for each trial of doping. "\
                        "Silver acetate solution of {AgConc_M} M is made with {AgMass_mg} mg of silver acetate in {MeOH_vol_mL} mL of MeOH and {H2O_vol_mL} mL of water. "\
                        "{Doping_pct} % Ag doping requires {AgSol_uL} uL of silver doping solution, respectively. "\
                        "The reaction was performed at 1000 rpm for {Time_min} minutes at {Temperature_C} oC. "\
                        "Fluorescence of each doped sample was collected as a 30x dilution with hexanes." \
                        "Knowing that: 1) an increase in silver doping will increase the fluorescence SECONDARY PEAK AREA, "\
                        "and 2) QUANTUM YIELD decreases as Ag doping increases, although some reports indicate that theres a slight increase before decreasing."
    elif version == 2:
        prompt_template = "Stock nanoplatelets have dimensions {Dimension1_nm} nm by {Dimension2_nm} nm and the optical density of 100x hexanes dilution of stock is {Dilution_pct}% "\
                        "recorded at 512nm. {Nano_stock_vol_mL} mL nanoplatelets stock is diluted 7x by hexanes using {Diluted_vol_mL} mL for each trial of doping. "\
                        "Silver acetate solution of {AgConc_M} M is made with {AgMass_mg} mg of silver acetate in {MeOH_vol_mL} mL of MeOH and {H2O_vol_mL} mL of water. "\
                        "{Doping_pct} % Ag doping requires {AgSol_uL} uL of silver doping solution, respectively. "\
                        "The reaction was performed at 1000 rpm for {Time_min} minutes at {Temperature_C} oC. "\
                        "Fluorescence of each doped sample was collected as a 30x dilution with hexanes." \
                        "Knowing that: 1) an increase in silver doping will increase the fluorescence SECONDARY PEAK AREA, "\
                        "and 2) QUANTUM YIELD decreases as Ag doping increases, although some reports indicate that theres a slight increase before decreasing."
    elif version == 3:
        counts = float(features["Dimension1_nm"]) * float(features["Dimension2_nm"]) * 8.8968 * 4 / 0.221448125
        Dimension1_nm = features["Dimension1_nm"]
        Dimension2_nm = features["Dimension2_nm"]
        Dilution_pct = float(features["Dilution_pct"])
        prompt_template = f"Stock nanoplatelets (NPL) have dimensions {Dimension1_nm} nm by {Dimension2_nm}, which gives a total Cd count per NPL as {counts} "\
                        f"nm and the optical density of 100x hexanes dilution of stock is {Dilution_pct}% recorded at 512nm, resulting in a theoretical NPL concentration of {31900000/Dilution_pct} M. "\
                        "{Nano_stock_vol_mL} mL nanoplatelets stock is  diluted 7x by hexanes using {Diluted_vol_mL} mL for each trial of doping. "\
                        "Silver acetate solution of {AgConc_M} M is made with {AgMass_mg} mg of silver acetate in {MeOH_vol_mL} mL of MeOH and {H2O_vol_mL} mL of water. "\
                        "{Doping_pct} % Ag doping requires {AgSol_uL} uL of silver doping solution, respectively. "\
                        "The reaction was performed at 1000 rpm for {Time_min} minutes at {Temperature_C} oC. Fluorescence of "\
                        "each doped sample was collected as a 30x dilution with hexanes."
        
    elif version == 4:
        counts = float(features["Dimension1_nm"]) * float(features["Dimension2_nm"]) * 8.8968 * 4 / 0.221448125
        Dilution_pct = float(features["Dilution_pct"])
        prompt_template = "Experiment Overview: Stock nanoplatelets (NPL) have dimensions {Dimension1_nm} nm by {Dimension2_nm} nm, resulting "\
                        f"in an estimated Cd count per NPL of {counts}. "\
                        f"The optical density of a 100x hexanes dilution of stock is {Dilution_pct}% at 512 nm, yielding a theoretical NPL concentration of {31900000/Dilution_pct} M. "\
                        "{Nano_stock_vol_mL} mL nanoplatelets stock is diluted 7x in hexanes to reach {Diluted_vol_mL} mL for each doping trial. Silver acetate solution ({AgConc_M} M) "\
                        "is prepared using {AgMass_mg} mg of silver acetate in {MeOH_vol_mL} mL MeOH and {H2O_vol_mL} mL water."\
                        "Objective: This experiment investigates the relationship between Ag doping and fluorescence characteristics. Specifically: "\
                        "An increase in silver doping typically elevates the secondary peak area of fluorescence. "\
                        "Quantum yield generally decreases with Ag doping, though slight initial increases are reported."
    elif version == 5:
        prompt_template = "Stock nanoplatelets have dimensions {Dimension1_nm} nm by {Dimension2_nm} nm and the optical density of 100x hexanes dilution of stock is {Dilution_pct}% "\
                        "recorded at 512nm. {Nano_stock_vol_mL} mL nanoplatelets stock is diluted 7x by hexanes using {Diluted_vol_mL} mL for each trial of doping. "\
                        "Silver acetate solution of {AgConc_M} M is made with {AgMass_mg} mg of silver acetate in {MeOH_vol_mL} mL of MeOH and {H2O_vol_mL} mL of water. "\
                        "{Doping_pct} % Ag doping requires {AgSol_uL} uL of silver doping solution, respectively. "\
                        "The reaction was performed at 1000 rpm for {Time_min} minutes at {Temperature_C} oC. "\
                        "Fluorescence of each doped sample was collected as a 30x dilution with hexanes." \
                        "Knowing that: 1) an increase in silver doping will increase the fluorescence SECONDARY PEAK AREA, "\
                        "2) QUANTUM YIELD decreases as Ag doping increases, although some reports indicate that theres a slight increase before decreasing, "\
                        "and 3) increasing the MeOH and Water quantities, which are incompatible solvents for the NPLs in hexanes, increases the solubility of Ag but also negatively impacts the QY of NPLs"
        
        
    #check if features are as dataframe, change to kwargs
    if isinstance(features, np.ndarray):
        features = features.tolist()
        return prompt_template.format(**features)
    features = features.to_dict()
    return prompt_template.format(**features)


In [7]:
#replace the column names with the prompt names Dinemsion1(nm) -> Dimension1_nm, etc
data.columns = [col.replace('(','_').replace(')','').replace('%','pct') for col in data.columns]
print(data.columns)

Index(['Dimension1_nm', 'Dimension2_nm', 'Dilution_pct', 'Nano_stock_vol_mL',
       'Diluted_vol_mL', 'AgConc_M', 'AgMass_mg', 'MeOH_vol_mL', 'H2O_vol_mL',
       'Doping_pct', 'AgSol_uL', 'Temperature_C', 'Time_min'],
      dtype='object')


In [10]:
#make prompt from features
prompts = []
for i in range(len(data)):
    prompt = make_prompt_from_features(data.iloc[i])
    prompts.append(prompt)

#save prompts in ; separeted file
with open('Data/pools_preprocessed.txt','w') as f:
    for prompt in prompts:
        f.write(prompt + '\n')

In [ ]:
import pandas as pd
procedures_df = pd.read_csv('/Data/processed_data.csv')
procedures_df.head()

,Dimension1(nm),Dimension2(nm),OD,Dilution(%),Nano_stock_vol(mL),Diluted_vol(mL),AgConc(M),AgMass(mg),MeOH_vol(mL),H2O_vol(mL),...,QY_peak1,QY_peak2,Fl_Wavelengths,Fluorescence,Fluorescence_norm,Abs_Wavelengths,Absorbance,Absorbance_norm,CIE_x,CIE_y
0,18.8,14.04,100x,0.175,0.25,1.75,0.0060,7.8,5,2,...,0.039617,3.074970e-01,"[441.0, 442.0, 443.0, 444.0, 445.0, 446.0, 447...","[5.3922465999834595, 5.551067438955388, 5.7155...","[0.0010700124103508776, 0.0011015280811517685,...","[300.0, 301.0, 302.0, 303.0, 304.0, 305.0, 306...","[0.43802025, 0.4338412, 0.42938566666666667, 0...","[1.0, 0.9904592310515324, 0.9802872507987169, ...",0.521398,0.447538
1,18.8,14.04,100x,0.175,0.25,1.75,0.0060,7.8,5,2,...,0.000864,2.307426e-01,"[441.0, 442.0, 443.0, 444.0, 445.0, 446.0, 447...","[0.05875342277612322, 0.061531410939200504, 0....","[2.1105257504146727e-05, 2.2103159460405937e-0...","[300.0, 301.0, 302.0, 303.0, 304.0, 305.0, 306...","[0.43802025, 0.4338412, 0.42938566666666667, 0...","[1.0, 0.9904592310515324, 0.9802872507987169, ...",0.650174,0.348096
2,18.8,14.04,100x,0.175,0.25,1.75,0.0060,7.8,5,2,...,0.001524,2.346841e-01,"[441.0, 442.0, 443.0, 444.0, 445.0, 446.0, 447...","[0.16726935860227754, 0.17430678840129743, 0.1...","[5.7619824046038625e-05, 6.00440305483188e-05,...","[300.0, 301.0, 302.0, 303.0, 304.0, 305.0, 306...","[0.43802025, 0.4338412, 0.42938566666666667, 0...","[1.0, 0.9904592310515324, 0.9802872507987169, ...",0.639103,0.358419
3,18.8,14.04,100x,0.175,0.25,1.75,0.0060,7.8,5,2,...,0.000248,8.388796e-02,"[441.0, 442.0, 443.0, 444.0, 445.0, 446.0, 447...","[0.0059593828942451415, 0.006355854469534446, ...","[5.1227292593135085e-06, 5.463539134306094e-06...","[300.0, 301.0, 302.0, 303.0, 304.0, 305.0, 306...","[0.43802025, 0.4338412, 0.42938566666666667, 0...","[1.0, 0.9904592310515324, 0.9802872507987169, ...",0.658767,0.339308
4,18.8,14.04,100x,0.175,0.25,1.75,0.0007,3.5,20,8,...,0.276434,4.928815e-08,"[441.0, 442.0, 443.0, 444.0, 445.0, 446.0, 447...","[1.9618019074826825e-16, 6.15887754421026e-16,...","[5.600871903107058e-22, 1.7583367648116504e-21...","[300.0, 301.0, 302.0, 303.0, 304.0, 305.0, 306...","[0.43802025, 0.4338412, 0.42938566666666667, 0...","[1.0, 0.9904592310515324, 0.9802872507987169, ...",0.041857,0.777612
